In [1]:
import os
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from PIL import Image
from vt_plug.dataset.utils import de_norm_box_xyxy_square2origin, bbox_to_wh_coco

[2025-03-07 20:31:45,244] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
gt_box_path = "/data/Aaronzhu/DatasetStage1/MSCOCO/2017/annotations/instances_val2017.json"
pred_save_path = "/code/vt_plug/work_dirs/ablation/0305_det_no_match/eval_full/coco_det_class_small.json"

In [3]:
all_file = []
with open(pred_save_path,'r') as f:
    for line in f:
        json_file = json.loads(line.strip())
        all_file.extend(json_file)
    f.close()

In [4]:
from tqdm import tqdm

all_file_processed = []
coco_val_image_dir = "/data/Aaronzhu/DatasetStage1/MSCOCO/2017/val2017"

for file in tqdm(all_file):
    image_id = str(file['image_id']).zfill(12)
    image = Image.open(os.path.join(coco_val_image_dir,f"{image_id}.jpg"))
    bbox = de_norm_box_xyxy_square2origin(file['bbox'],image.width,image.height)
    bbox = bbox_to_wh_coco(bbox)
    all_file_processed.append(
        {'image_id':file['image_id'],
         'category_id':file['category_id'],
         'bbox':bbox,
         'score':file['score']}
    )

100%|██████████| 36211/36211 [00:16<00:00, 2151.34it/s]


In [5]:
from collections import defaultdict

all_items = defaultdict(list)
for i,item in enumerate(all_file_processed):
    image_id = str(item['image_id']).zfill(12)
    item['index'] = i
    all_items[image_id].append(item)


with open(gt_box_path,'r') as f:
    gt_box = json.load(f)
    f.close()

all_gt_items = defaultdict(list)
for i,item in enumerate(gt_box['annotations']):
    image_id = str(item['image_id']).zfill(12)
    item['index'] = i
    all_gt_items[image_id].append(item)


all_images_items = defaultdict(list)
for i,item in enumerate(gt_box['images']):
    image_id = str(item['id']).zfill(12)
    item['index'] = i
    all_images_items[image_id].append(item)

In [6]:
def retain_common_keys(dict_a, dict_b, dict_c):
    # 找到两个字典的键的交集
    common_keys = dict_a.keys() & dict_b.keys() & dict_c.keys()
    removed_keys_a = dict_a.keys() - common_keys
    removed_keys_b = dict_b.keys() - common_keys
    removed_keys_c = dict_c.keys() - common_keys
    
    # 根据交集过滤两个字典
    filtered_dict_a = {key: dict_a[key] for key in common_keys}
    filtered_dict_b = {key: dict_b[key] for key in common_keys}
    filtered_dict_c = {key: dict_c[key] for key in common_keys}
    
    return filtered_dict_a, filtered_dict_b, filtered_dict_c, removed_keys_a, removed_keys_b, removed_keys_c


all_items, all_gt_items, all_images_items, removed_keys_a, removed_keys_b, removed_keys_c = retain_common_keys(all_items, all_gt_items, all_images_items)
print(len(gt_box['images']))
print(len(gt_box['annotations']))
print(len(all_images_items.keys()))
print(len(all_items.keys()))
print(len(all_gt_items.keys()))

gt_box['images'] = [item for item in gt_box['images'] if item['id'] not in list(removed_keys_c)]
# new_images = []
# for item in gt_mask['images']:
#     if item['id'] not in list(removed_keys_b):
#         new_images.append(item)
# gt_mask['images'] = new_images


new_annotations = []
# for item in gt_mask['annotations']:
#     if item['image_id'] not in list(removed_keys_b):
#         new_annotations.append(item)
# gt_mask['annotations'] = new_annotations
for key in all_gt_items.keys():
    annotation = all_gt_items[key]
    new_annotations.extend(annotation)
gt_box['annotations'] = new_annotations


# gt_mask['annotations'] = [item for item in gt_mask['annotations'] if item['image_id'] not in list(removed_keys_b)]
print(len(gt_box['images']))
print(len(gt_box['annotations']))

5000
36781
4858
4858
4858
5000
35985


In [7]:
def find_duplicate_ids(data):
    seen_ids = set()
    duplicate_ids = set()

    # 遍历列表，检查每个id是否重复
    for item in data:
        if item['id'] in seen_ids:
            duplicate_ids.add(item['id'])
        else:
            seen_ids.add(item['id'])
    
    # 如果有重复的id，打印出来
    if duplicate_ids:
        print(f"重复的 id: {duplicate_ids}")
    else:
        print("没有重复的 id")

find_duplicate_ids(gt_box['images'])

没有重复的 id


In [8]:
tmp_save_dir = "/code/vt_plug/tmp"
split = 'val'

# all_file = all_file[:16955]

with open(f"{tmp_save_dir}/box_coco2017_{split}.json",'w') as f:
    json.dump(all_file_processed,f)
    f.close()

with open(f"{tmp_save_dir}/box_gt_coco2017_{split}.json",'w') as f2:
    json.dump(gt_box,f2)
    f2.close()

pred_save_path = f"{tmp_save_dir}/box_coco2017_{split}.json"
gt_mask_path = f"{tmp_save_dir}/box_gt_coco2017_{split}.json"

coco_gt = COCO(gt_mask_path)
coco_dt = coco_gt.loadRes(pred_save_path)  # load predictions
# Initialize COCOEval and specify the metric you want to use
coco_eval = COCOeval(coco_gt, coco_dt, "bbox")  # "segm" for segmentation  bbox for detection
# Evaluate on a specific category
# coco_eval.params.catIds = [1]  # your category ID
# Evaluate
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

loading annotations into memory...
Done (t=0.66s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=8.95s).
Accumulating evaluation results...
DONE (t=1.66s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.079
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.049
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets